# <b>Introduction<b>

In this project, I classify Yelp round-10 review datasets. The reviews contain a lot of metadata that can be mined and used to infer meaning, business attributes, and sentiment. For simplicity, I classify the review comments into two class: either as positive or negative. Reviews that have star higher than three are regarded as positive while the reviews with star less than or equal to 3 are negative. Therefore, the problem is a supervised learning. To build and train the model, I first tokenize the text and convert them to sequences. Each review comment is limited to 50 words. As a result, short texts less than 50 words are padded with zeros, and long ones are truncated. After processing the review comments, I trained three model in three different ways: 

<li> Model-1: In this model, a neural network with LSTM and a single embedding layer were used. 
<li> Model-2: In Model-1, an extra 1D convolutional layer has been added on top of LSTM layer to reduce the training time.
<li> Model-3:  In this model, I use the same network architecture as Model-2, but use the pre-trained glove 100 dimension word embeddings as initial input.

Since there are about 1.6 million input comments, it takes a while to train the models. To reduce the training time step, I limit the training epoch to three. After three epochs, it is evident that Model-2 is better regarding both training time and validation accuracy.

## <b>Project Outline <b>

In this project I will cover the follwouings :

<li> Download data from yelp and process them
<li> Build neural network with LSTM
<li> Build neural network with LSTM and CNN
<li> Use pre-trained GloVe word embeddings
<li> Word Embeddings from Word2Vec

## <b>Import libraries<b>

In [1]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2joon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Import Data

In [2]:
import wget
# wget.download('https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz')

In [3]:
# unzip manually

## <b> Data Processing<b>

In [7]:
# df = pd.read_csv('train.csv', sep = '|', , error_bad_lines=False)
df = pd.read_csv('yelp_review_polarity_csv/test.csv',names = ['stars', 'text'])

In [8]:
df.head()

,stars,text
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [9]:
# df = df.dropna()
# df = df[df.stars.apply(lambda x: x.isnumeric())]
# df = df[df.stars.apply(lambda x: x !="")]
# df = df[df.text.apply(lambda x: x !="")]

In [10]:
# df.describe()

In [11]:
df.head()

,stars,text
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


### Convert five classes into two classes (positive = 1 and negative = 0)

Since the main purpose is to identify positive or negative comments, I convert five class star category into two classes: 

<li> (1) Positive: comments with stars > 3 and 
<li> (2) Negative: comments with stars <= 3

In [12]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

### Tokenize text data

Because of the computational expenses, I use the top 20000 unique words. First, tokenize the comments then convert those into sequences. I keep 50 words to limit the number of words in each comment. 

In [13]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [14]:
df['text'] = df['text'].map(lambda x: clean_text(x))

In [15]:
df.head(10)

,stars,text
0,2,contrari review zero complaint servic price ge...
1,1,last summer appoint get new tire wait super lo...
2,2,friend staff starbuck fair get anywher els som...
3,1,food good unfortun servic hit miss main issu s...
4,2,even car filen basement worth bus trip waterfr...
5,2,pictur billi joel piano man doubl mix beer row...
6,1,mediocr servic cold food ! food wait long lett...
7,1,ok ! let tell bad experi first went d b last n...
8,1,use love d b first open waterfront gone hill y...
9,2,like barn nobl nice comfi cafe larg select boo...


In [16]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text'])

sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [17]:
print(data.shape)

(38000, 50)


## <b>Build neural network with LSTM<b>

### Network Architechture

The network starts with an embedding layer. The layer lets the system expand each token to a more massive vector, allowing the network to represent a word in a meaningful way. The layer takes 20000 as the first argument, which is the size of our vocabulary, and 100 as the second input parameter, which is the dimension of the embeddings. The third parameter is the input_length of 50, which is the length of each comment sequence.

In [18]:
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=50))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the network

There are about 1.6 million comments, and it takes a while to train the model in a MacBook Pro. To save time I have used only three epochs. GPU machines can be used to accelerate the training with more time steps. I split the whole datasets as 60% for training and 40% for validation.

In [19]:
model_lstm.fit(data, np.array(labels), validation_split=0.4, epochs=3)

Epoch 1/3
713/713 [==============================] - 77s 102ms/step - loss: 0.0098 - accuracy: 0.9992 - val_loss: 1.3967e-05 - val_accuracy: 1.0000
Epoch 2/3
713/713 [==============================] - 75s 106ms/step - loss: 9.1730e-06 - accuracy: 1.0000 - val_loss: 5.6180e-06 - val_accuracy: 1.0000
Epoch 3/3
713/713 [==============================] - 75s 105ms/step - loss: 4.2696e-06 - accuracy: 1.0000 - val_loss: 2.9942e-06 - val_accuracy: 1.0000


##  <b>Build neural network with LSTM and CNN <b>
The LSTM model worked well. However, it takes forever to train three epochs. One way to speed up the training time is to improve the network adding “Convolutional” layer. Convolutional Neural Networks (CNN) come from image processing. They pass a “filter” over the data and calculate a higher-level representation. They have been shown to work surprisingly well for text, even though they have none of the sequence processing ability of LSTMs.

In [20]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv 

In [21]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Epoch 1/3
713/713 [==============================] - 35s 44ms/step - loss: 0.0080 - accuracy: 0.9986 - val_loss: 5.3594e-06 - val_accuracy: 1.0000
Epoch 2/3
713/713 [==============================] - 30s 42ms/step - loss: 3.9978e-06 - accuracy: 1.0000 - val_loss: 3.0569e-06 - val_accuracy: 1.0000
Epoch 3/3
713/713 [==============================] - 30s 41ms/step - loss: 2.4588e-06 - accuracy: 1.0000 - val_loss: 1.9915e-06 - val_accuracy: 1.0000


### Save processed Data

In [22]:
df_save = pd.DataFrame(data)
df_label = pd.DataFrame(np.array(labels))

In [23]:
result = pd.concat([df_save, df_label], axis = 1)

In [24]:
result.to_csv('train_dense_word_vectors.csv', index=False)

## <b>Word embedding visialization<b>

In this subsection, I want to visualize word embedding weights obtained from trained models. Word embeddings with 100 dimensions are first reduced to 2 dimensions using t-SNE. Tensorflow has an excellent tool to visualize the embeddings in a great way, but here I just want to visualize the word relationship. 

### Get embedding weights from glove

In [26]:
lstm_embds = model_lstm.layers[0].get_weights()[0]

In [27]:
conv_embds = model_conv.layers[0].get_weights()[0]

### Get word list 

In [28]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

### Scatter plot of first two components of TSNE

In [29]:
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)

#### 1. LSTM

In [31]:
number_of_words = 100
lstm_tsne_embds = TSNE(n_components=2).fit_transform(lstm_embds)

In [32]:
plot_words(lstm_tsne_embds, 0, number_of_words, 1)

#### 2. CNN + LSTM

In [33]:
conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)

KeyboardInterrupt: 

In [ ]:
plot_words(conv_tsne_embds, 0, number_of_words, 1)

## <b>Word Embeddings from Word2Vec<b>

In this subsection, I use word2vec to create word embeddings from the review comments. Word2vec is one algorithm for learning a word embedding from a text corpus.

In [37]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2joon\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

###  Tokenize the reviews coments. 

In [38]:
df['tokenized'] = df.apply(lambda row : nltk.word_tokenize(row['text']), axis=1)

In [39]:
df.head()

,stars,text,tokenized
0,2,contrari review zero complaint servic price ge...,"[contrari, review, zero, complaint, servic, pr..."
1,1,last summer appoint get new tire wait super lo...,"[last, summer, appoint, get, new, tire, wait, ..."
2,2,friend staff starbuck fair get anywher els som...,"[friend, staff, starbuck, fair, get, anywher, ..."
3,1,food good unfortun servic hit miss main issu s...,"[food, good, unfortun, servic, hit, miss, main..."
4,2,even car filen basement worth bus trip waterfr...,"[even, car, filen, basement, worth, bus, trip,..."


### Train word2vec model

In [41]:
model_w2v = Word2Vec(df['tokenized'])#, size=100)

In [42]:
X = model_w2v[model_w2v.wv.vocab]

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

### Plot Word Vectors Using PCA

In [43]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
tsvd = TruncatedSVD(n_components=5, n_iter=10)
result = tsvd.fit_transform(X)

In [ ]:
result.shape

In [ ]:
tsvd_word_list = []
words = list(model_w2v.wv.vocab)
for i, word in enumerate(words):
    tsvd_word_list.append(word)

trace = go.Scatter(
    x = result[0:number_of_words, 0], 
    y = result[0:number_of_words, 1],
    mode = 'markers',
    text= tsvd_word_list[0:number_of_words]
)

layout = dict(title= 'SVD 1 vs SVD 2',
              yaxis = dict(title='SVD 2'),
              xaxis = dict(title='SVD 1'),
              hovermode= 'closest')

fig = dict(data = [trace], layout= layout)
py.iplot(fig)